# Cleaning f1db data

This notebook is to clean the series of csv tables coming from the [Ergast Developer API](https://ergast.com/mrd/db/). I'm downloading the raw csv tables and processing them here.

Some things to note about the data:

* The data is not chronologically organized in the csv case. Race numbering starts with 1 corresponding to 2007's first race, and then goes back at a point to fill in the 1950–2006 data and also add in the data moving forward.
* In the standings table, a new entry is not added after a race if the driver does not change standings.
* When a cell is `\N` it means the cell is null, or the value wasn't recorded. This largely applies to things for which we had no data in the beginning.
* Looking at the original `raceId`, something happened because Azerbaijan GP 2019 was the 1001st race but their id for the same race was 1013. I assume the counting was pushed ahead somewhere in the middle, but things are still okay.
* The `drivers` table is actually named `driver.csv`.


**I last downloaded this data on 19-06-10.**

In [1]:
import pandas as pd
import numpy as np

First thing we'll do is add headers to each table based on this [schema text file](http://ergast.com/schemas/f1db_schema.txt). I will also couple removing the `\N` characters in this step.

In [2]:
baseurl = "../data/f1db_csv/"
destination = "../data/working/"

#names correspond to the csv files and not the table names given at the top of the schema text file
tables = ["circuits", "constructor_results", "constructor_standings", "constructors", "driver_standings", 
          "driver", "lap_times", "pit_stops", "qualifying", "races", "results", "seasons", "status"]

# all the headers are grouped by table and put into one big list
headers = [["circuitId", "circuitRef", "name", "location", "country","lat", "lng", "alt", "url"], #circuits
          ["constructorResultsId", "raceId", "constructorId", "points", "status"], #constructor_results
          ["constructorStandingsId", "constructorId", "points", "position", "positionText", "wins"], #constructor_standings
          ["constructorId", "constructorRef", "name", "nationality", "url"], #constructors
          ["driverStandingsId", "raceId", "driverId", "points", "position", "positionText", "wins"], #driver_standings
          ["driverId", "driverRef", "number", "code", "forename", "surname", "dob", "nationality", "url" ], #drivers
          ["raceId", "driverId", "lap", "position", "time", "milliseconds"], #lap_times
          ["raceId", "driverId", "stop", "lap", "time", "duration", "milliseconds"], #pit_stops
          ["qualifyId", "raceId", "driverId", "constructorId", "number",
           "position", "q1", "q2","q3"], #qualifying
          ["raceId", "year", "round", "circuitId", "name", "date", "time", "url"], #races
          ["resultId", "raceId", "driverId", "constructorId", "number", "grid", "position",
           "positionText", "positionOrder", "points", "laps", "time", "milliseconds",
           "fastestLap", "rank", "fastestLapTime", "fastestLapSpeed", "statusId"], #results
          ["year","url"], #seasons
          ["statusId", "status"] #status
          ]

for i in range(len(tables)):
    #pick the right table name and the right header
    table = tables[i]
    header = headers[i]
    
    #import the csv file associated with the table and add in the header to it
    df = pd.read_csv(baseurl+table+".csv",names=header)
    #get rid of the null characters
    df = df.replace("\\N","")
    #export back to a csv in the working folder
    df.to_csv(destination+table+".csv", index=False, mode="w+")
    
    #status updates
    print(table,"done", sep=" --> ")

circuits --> done
constructor_results --> done
constructor_standings --> done
constructors --> done
driver_standings --> done
driver --> done
lap_times --> done
pit_stops --> done
qualifying --> done
races --> done
results --> done
seasons --> done
status --> done


I could try to figure out how to sort the drivers chronologically and the races as well as I had done before, but I don't think that's actually as important to my analysis.

---
Any added cleaning that needs to happen will happen here

---

## Getting a "master" table to work with

I'm going to do is create a "master" table with all the relevant columns I think are of interest.

In [3]:
results = pd.read_csv("../data/working/results.csv")

In [4]:
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,+5.478,5696094.0,41.0,3.0,1:27.739,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,+8.163,5698779.0,41.0,5.0,1:28.090,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,+17.181,5707797.0,58.0,7.0,1:28.603,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,+18.014,5708630.0,43.0,1.0,1:27.418,218.385,1


In [5]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24340 entries, 0 to 24339
Data columns (total 18 columns):
resultId           24340 non-null int64
raceId             24340 non-null int64
driverId           24340 non-null int64
constructorId      24340 non-null int64
number             24334 non-null float64
grid               24340 non-null int64
position           13690 non-null float64
positionText       24340 non-null object
positionOrder      24340 non-null int64
points             24340 non-null float64
laps               24340 non-null int64
time               6269 non-null object
milliseconds       6268 non-null float64
fastestLap         5922 non-null float64
rank               6091 non-null float64
fastestLapTime     5922 non-null object
fastestLapSpeed    5922 non-null float64
statusId           24340 non-null int64
dtypes: float64(7), int64(8), object(3)
memory usage: 3.3+ MB


We're starting with the results table. Columns on this I'm interested in:

* **raceId**: the unique Id for the race
* **driverId**: the unique Id for the driver
* **constructorId**: the unique Id for the constructor(team)
* **grid**: the driver's grid position (where they start the race)
* **position**: the driver's finishing position as an float if the driver finished the race. 
* **positionText**: the driver's finishing position as text. This gives us more information about whether the driver finished or not.  
* **positionOrder**: the driver's finishing position in order from first to last for a given race. Usually the driver who retires first in the race is last and then the orders go up. Ordering takes into account if the driver withdrew, failed to qualify, was excluded from the final positions, or other such situations. 
* **points**: the number of points the driver got. Formula1 has changed the point systems several times since 1950. In this case points are awarded based on the season of the race.
* **status**: information about the driver's finishing position. Wether they finished, retired, withdrew, had a collission, etc. It can also give us information about the car component that went wrong and caused a retirement.

To this subset I want to add more information about the race, the driver and the constructor. I'll begin with the race information.

Before that I'll make a copy of the results table to which I will add things.

In [6]:
master = results[["raceId", "driverId", "constructorId", "grid", "position", "positionText","positionOrder", "points", "statusId"]]

In [7]:
master.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,statusId
0,18,1,1,1,1.0,1,1,10.0,1
1,18,2,2,5,2.0,2,2,8.0,1
2,18,3,3,7,3.0,3,3,6.0,1
3,18,4,4,11,4.0,4,4,5.0,1
4,18,5,1,3,5.0,5,5,4.0,1


### Adding more race information

In [8]:
races = pd.read_csv("../data/working/races.csv")

In [9]:
races.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [10]:
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 8 columns):
raceId       1018 non-null int64
year         1018 non-null int64
round        1018 non-null int64
circuitId    1018 non-null int64
name         1018 non-null object
date         1018 non-null object
time         287 non-null object
url          1018 non-null object
dtypes: int64(4), object(4)
memory usage: 63.7+ KB


From the races table, I care about the `year`, `round`, `name` columns.

In [11]:
races_slice = races[["raceId", "year", "round", "name"]]

Changing the name of the `name` column so it's more clear that it's the name of the grand prix:

In [12]:
races_slice = races_slice.rename(columns = {"name": "prixName"})

In [13]:
races_slice.head()

,raceId,year,round,prixName
0,1,2009,1,Australian Grand Prix
1,2,2009,2,Malaysian Grand Prix
2,3,2009,3,Chinese Grand Prix
3,4,2009,4,Bahrain Grand Prix
4,5,2009,5,Spanish Grand Prix


I will now add this slice of the race information data to the master table:

In [14]:
master = master.merge(races_slice, on="raceId")

In [15]:
master.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,statusId,year,round,prixName
0,18,1,1,1,1.0,1,1,10.0,1,2008,1,Australian Grand Prix
1,18,2,2,5,2.0,2,2,8.0,1,2008,1,Australian Grand Prix
2,18,3,3,7,3.0,3,3,6.0,1,2008,1,Australian Grand Prix
3,18,4,4,11,4.0,4,4,5.0,1,2008,1,Australian Grand Prix
4,18,5,1,3,5.0,5,5,4.0,1,2008,1,Australian Grand Prix


### Adding more driver information

In [16]:
drivers = pd.read_csv("../data/working/driver.csv")

In [17]:
drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [18]:
drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 9 columns):
driverId       847 non-null int64
driverRef      847 non-null object
number         44 non-null float64
code           90 non-null object
forename       847 non-null object
surname        847 non-null object
dob            847 non-null object
nationality    847 non-null object
url            846 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 59.6+ KB


I'm interested in the `forename`, and `surname` columns.

In [19]:
drivers_slice = drivers[["driverId", "forename", "surname"]]

In [20]:
drivers_slice.head()

,driverId,forename,surname
0,1,Lewis,Hamilton
1,2,Nick,Heidfeld
2,3,Nico,Rosberg
3,4,Fernando,Alonso
4,5,Heikki,Kovalainen


In [21]:
master = master.merge(drivers_slice, on="driverId")

In [22]:
master.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,statusId,year,round,prixName,forename,surname
0,18,1,1,1,1.0,1,1,10.0,1,2008,1,Australian Grand Prix,Lewis,Hamilton
1,19,1,1,9,5.0,5,5,4.0,1,2008,2,Malaysian Grand Prix,Lewis,Hamilton
2,20,1,1,3,13.0,13,13,0.0,11,2008,3,Bahrain Grand Prix,Lewis,Hamilton
3,21,1,1,5,3.0,3,3,6.0,1,2008,4,Spanish Grand Prix,Lewis,Hamilton
4,22,1,1,3,2.0,2,2,8.0,1,2008,5,Turkish Grand Prix,Lewis,Hamilton


### Adding constructor information

In [23]:
constructors = pd.read_csv("../data/working/constructors.csv")

In [24]:
constructors.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [25]:
constructors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 5 columns):
constructorId     209 non-null int64
constructorRef    209 non-null object
name              209 non-null object
nationality       209 non-null object
url               209 non-null object
dtypes: int64(1), object(4)
memory usage: 8.2+ KB


For these, I'm interested in both the `constructorRef`, a shortname for each constructor, and the constructor `name`.

In [26]:
constructors_slice = constructors[["constructorId", "constructorRef", "name"]]

Changing the name of the `name` column so it is more clear that it's the name of the constructor

In [27]:
constructors_slice = constructors_slice.rename(columns = {"name": "constructor"})

In [28]:
constructors_slice.head()

,constructorId,constructorRef,constructor
0,1,mclaren,McLaren
1,2,bmw_sauber,BMW Sauber
2,3,williams,Williams
3,4,renault,Renault
4,5,toro_rosso,Toro Rosso


In [29]:
master = master.merge(constructors_slice, on="constructorId")

In [30]:
master.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,statusId,year,round,prixName,forename,surname,constructorRef,constructor
0,18,1,1,1,1.0,1,1,10.0,1,2008,1,Australian Grand Prix,Lewis,Hamilton,mclaren,McLaren
1,19,1,1,9,5.0,5,5,4.0,1,2008,2,Malaysian Grand Prix,Lewis,Hamilton,mclaren,McLaren
2,20,1,1,3,13.0,13,13,0.0,11,2008,3,Bahrain Grand Prix,Lewis,Hamilton,mclaren,McLaren
3,21,1,1,5,3.0,3,3,6.0,1,2008,4,Spanish Grand Prix,Lewis,Hamilton,mclaren,McLaren
4,22,1,1,3,2.0,2,2,8.0,1,2008,5,Turkish Grand Prix,Lewis,Hamilton,mclaren,McLaren


### Adding status information

In [31]:
status = pd.read_csv("../data/working/status.csv")

In [32]:
status.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [33]:
master = master.merge(status, on="statusId")

In [34]:
master.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,statusId,year,round,prixName,forename,surname,constructorRef,constructor,status
0,18,1,1,1,1.0,1,1,10.0,1,2008,1,Australian Grand Prix,Lewis,Hamilton,mclaren,McLaren,Finished
1,19,1,1,9,5.0,5,5,4.0,1,2008,2,Malaysian Grand Prix,Lewis,Hamilton,mclaren,McLaren,Finished
2,21,1,1,5,3.0,3,3,6.0,1,2008,4,Spanish Grand Prix,Lewis,Hamilton,mclaren,McLaren,Finished
3,22,1,1,3,2.0,2,2,8.0,1,2008,5,Turkish Grand Prix,Lewis,Hamilton,mclaren,McLaren,Finished
4,23,1,1,3,1.0,1,1,10.0,1,2008,6,Monaco Grand Prix,Lewis,Hamilton,mclaren,McLaren,Finished


---

Now that my master dataFrame is set, let me rearrange the columns and save it to a csv file to work with in the analysis. Some columns—`statusId`, `constructorId`—are not needed.

In [35]:
master = master[["raceId", "year", "round", "prixName","constructor","constructorRef", "forename", "surname", "grid", "position", "positionText", "positionOrder", "points","status"]]

In [36]:
master.head()

,raceId,year,round,prixName,constructor,constructorRef,forename,surname,grid,position,positionText,positionOrder,points,status
0,18,2008,1,Australian Grand Prix,McLaren,mclaren,Lewis,Hamilton,1,1.0,1,1,10.0,Finished
1,19,2008,2,Malaysian Grand Prix,McLaren,mclaren,Lewis,Hamilton,9,5.0,5,5,4.0,Finished
2,21,2008,4,Spanish Grand Prix,McLaren,mclaren,Lewis,Hamilton,5,3.0,3,3,6.0,Finished
3,22,2008,5,Turkish Grand Prix,McLaren,mclaren,Lewis,Hamilton,3,2.0,2,2,8.0,Finished
4,23,2008,6,Monaco Grand Prix,McLaren,mclaren,Lewis,Hamilton,3,1.0,1,1,10.0,Finished


I'm considering adding some new columns to clarify and combine fields:
1. one that has the full name of the driver `(forename + surname)`
1. one that has the full name of the grand prix `(year + prixName)`

I will leave this for later because it doesn't seem as necessary right now (and I think I can do it when I'm working with D3).

I will go ahead and sort the table by `year` > `round` > `positionOrder` so the first entry is the first position of the first race in 1950 and the last entry is the last position of the most recent race.

In [37]:
master = master.sort_values(by=["year", "round","positionOrder"])

In [38]:
master.head()

,raceId,year,round,prixName,constructor,constructorRef,forename,surname,grid,position,positionText,positionOrder,points,status
3886,833,1950,1,British Grand Prix,Alfa Romeo,alfa,Nino,Farina,1,1.0,1,1,9.0,Finished
3893,833,1950,1,British Grand Prix,Alfa Romeo,alfa,Luigi,Fagioli,2,2.0,2,2,6.0,Finished
3891,833,1950,1,British Grand Prix,Alfa Romeo,alfa,Reg,Parnell,4,3.0,3,3,4.0,Finished
17599,833,1950,1,British Grand Prix,Talbot-Lago,lago,Yves,Cabantous,6,4.0,4,4,3.0,+2 Laps
17596,833,1950,1,British Grand Prix,Talbot-Lago,lago,Louis,Rosier,9,5.0,5,5,2.0,+2 Laps


In [39]:
master.tail()

,raceId,year,round,prixName,constructor,constructorRef,forename,surname,grid,position,positionText,positionOrder,points,status
16219,1016,2019,7,Canadian Grand Prix,Williams,williams,George,Russell,18,16.0,16,16,0.0,+2 Laps
17346,1016,2019,7,Canadian Grand Prix,Haas F1 Team,haas,Kevin,Magnussen,0,17.0,17,17,0.0,+2 Laps
21233,1016,2019,7,Canadian Grand Prix,Williams,williams,Robert,Kubica,19,18.0,18,18,0.0,+3 Laps
16057,1016,2019,7,Canadian Grand Prix,Toro Rosso,toro_rosso,Alexander,Albon,13,NaN,R,19,0.0,Retired
16019,1016,2019,7,Canadian Grand Prix,McLaren,mclaren,Lando,Norris,8,NaN,R,20,0.0,Retired


Ready for saving:

In [40]:
master.to_csv("../data/working/master_results.csv", index=False, mode="w+")